# MLKDE: Estimação de Densidade e Detecção de Outliers em Preços do Airbnb em Lisboa

**Objetivo:** mostrar de forma rápida o que é KDE, como a largura de banda pode ser escolhida por máxima verossimilhança leave-one-out (MLKDE) e aplicar ao dataset do Airbnb em Lisboa para entender a distribuição de preços e destacar outliers.

## Dataset em 1 minuto
- Dados públicos do [Inside Airbnb](http://insideairbnb.com/get-the-data/) para Lisboa.
- Usaremos apenas a coluna **price** e criaremos `log_price` para lidar com a cauda longa.
- O download é automático: pega sempre o arquivo mais recente e salva em `data/listings_lisboa.csv`.

In [ ]:
import gzip
import io
import re
import shutil
import urllib.request
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from KDEpy import bw_selection

plt.style.use("seaborn-v0_8")


def download_latest_lisbon_listing(destination: Path) -> Path:
    destination.parent.mkdir(parents=True, exist_ok=True)
    if destination.exists():
        print(f"Arquivo já encontrado em {destination}. Usando versão local.")
        return destination

    index_url = "http://insideairbnb.com/get-the-data/"
    with urllib.request.urlopen(index_url, timeout=15) as resp:
        html = resp.read().decode("utf-8", errors="ignore")

    match = re.search(
        r"https://data\\.insideairbnb\\.com/portugal/lisbon/lisbon/[0-9-]+/data/listings\\.csv\\.gz",
        html,
    )
    if not match:
        raise RuntimeError("Não foi possível localizar o link de Lisboa no Inside Airbnb.")

    download_url = match.group(0)
    request = urllib.request.Request(download_url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(request, timeout=60) as resp:
        compressed_bytes = io.BytesIO(resp.read())

    with gzip.open(compressed_bytes, "rt", encoding="utf-8") as gz, destination.open("w", encoding="utf-8") as out:
        shutil.copyfileobj(gz, out)

    print(f"Arquivo baixado de {download_url}")
    print(f"Salvo como {destination}")
    return destination


data_path = Path("data/listings_lisboa.csv")
try:
    download_latest_lisbon_listing(data_path)
except Exception as exc:
    raise RuntimeError(
        "Não foi possível baixar o dataset automaticamente. Baixe o listings.csv de Lisboa no Inside Airbnb e salve como data/listings_lisboa.csv."
    ) from exc

raw_df = pd.read_csv(data_path)
prices = (
    raw_df["price"]
    .astype(str)
    .str.replace(r"[^0-9,\\.]", "", regex=True)
    .str.replace(",", "", regex=False)
)
prices = pd.to_numeric(prices, errors="coerce")

clean_df = raw_df.copy()
clean_df["price"] = prices
clean_df = clean_df.dropna(subset=["price"]).copy()
clean_df = clean_df[clean_df["price"] > 0].copy()
clean_df["log_price"] = np.log(clean_df["price"])

clean_df[["price", "log_price"]].head()

## KDE e a função objetivo (rapidinho)
- **KDE**: jeito de desenhar uma curva suave que representa a distribuição dos dados, sem assumir forma prévia.
- Fórmula (kernel Gaussiano): \(\hat f_h(x) = \frac{1}{n h} \sum_{i=1}^n K\!\left(\frac{x - x_i}{h}\right)\), com \(K\) Gaussiano e **h** controlando a suavidade.
- **MLKDE**: para cada \(h\), calculamos a log-verossimilhança leave-one-out (quão prováveis os dados ficam sob a curva) e escolhemos o \(h\) que a maximiza.

In [ ]:
def gaussian_kernel(u: np.ndarray) -> np.ndarray:
    return (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * u ** 2)


def kde_density(x_grid: np.ndarray, data: np.ndarray, bandwidth: float) -> np.ndarray:
    scaled = (x_grid[:, None] - data[None, :]) / bandwidth
    return gaussian_kernel(scaled).mean(axis=1) / bandwidth


def loo_log_likelihood(data: np.ndarray, bandwidth: float) -> float:
    if bandwidth <= 0:
        return -np.inf
    n = data.shape[0]
    scaled = (data[:, None] - data[None, :]) / bandwidth
    kernels = gaussian_kernel(scaled)
    np.fill_diagonal(kernels, 0.0)
    density_i = kernels.sum(axis=1) / ((n - 1) * bandwidth)
    density_i = np.maximum(density_i, 1e-12)
    return float(np.sum(np.log(density_i)))


log_prices = clean_df["log_price"].to_numpy()
std_log = np.std(log_prices, ddof=1) if len(log_prices) > 1 else 1.0

h_min = max(0.05, 0.1 * std_log)
h_max = max(h_min * 2, 1.5 * std_log)
h_grid = np.linspace(h_min, h_max, 40)

log_liks = [loo_log_likelihood(log_prices, h) for h in h_grid]
best_idx = int(np.argmax(log_liks))
h_mlkde = h_grid[best_idx]

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(h_grid, log_liks, label="Log-verossimilhança LOO")
ax.axvline(h_mlkde, color="crimson", linestyle="--", label=f"h ótimo = {h_mlkde:.3f}")
ax.set_xlabel("Bandwidth h")
ax.set_ylabel("Log-verossimilhança LOO")
ax.set_title("Busca do bandwidth por MLKDE")
ax.legend()
plt.show()

O pico indica o **h** que deixa os dados mais prováveis segundo o KDE. É esse valor (linha vermelha) que usaremos adiante.

In [ ]:
x_grid = np.linspace(log_prices.min() - 0.5, log_prices.max() + 0.5, 400)
mlkde_density = kde_density(x_grid, log_prices, h_mlkde)

fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(log_prices, bins=40, density=True, alpha=0.4, color="steelblue", edgecolor="white")
ax.plot(x_grid, mlkde_density, color="crimson", linewidth=2, label="KDE (h ótimo)")
ax.set_title("Distribuição de log(preço) com curva KDE (h ótimo por MLKDE)")
ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade")
ax.legend()
plt.show()

price_grid = np.linspace(clean_df["price"].quantile(0.01), clean_df["price"].quantile(0.99), 400)
log_grid = np.log(price_grid)
price_density = kde_density(log_grid, log_prices, h_mlkde) / price_grid

fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(clean_df["price"], bins=40, density=True, alpha=0.4, color="seagreen", edgecolor="white")
ax.plot(price_grid, price_density, color="darkgreen", linewidth=2, label="KDE em preços")
ax.set_title("Preços em escala original")
ax.set_xlabel("Preço por noite (EUR)")
ax.set_ylabel("Densidade")
ax.legend()
plt.show()

A maior parte dos valores se concentra em uma faixa intermediária; a curva KDE suaviza o histograma e destaca a cauda longa.

In [ ]:
from typing import Sequence


def lscv_objective(data: np.ndarray, bandwidth: float) -> float:
    if bandwidth <= 0:
        return np.inf
    n = data.size
    diff = data[:, None] - data[None, :]
    term1 = np.exp(-(diff ** 2) / (4 * bandwidth ** 2)).sum()
    term1 = term1 / (n ** 2 * bandwidth * np.sqrt(4 * np.pi))

    gauss = np.exp(-(diff ** 2) / (2 * bandwidth ** 2))
    off_sum = gauss.sum() - np.diag(gauss).sum()
    term2 = (2 / (n * (n - 1) * bandwidth * np.sqrt(2 * np.pi))) * off_sum
    return term1 - term2


def score_methods(data: np.ndarray, candidates: Sequence[float]):
    loglik = [loo_log_likelihood(data, h) for h in candidates]
    best_h = candidates[int(np.argmax(loglik))]
    h_lscv_candidates = np.linspace(h_min, h_max, 50)
    lscv_vals = [lscv_objective(data, h) for h in h_lscv_candidates]
    h_lscv = h_lscv_candidates[int(np.argmin(lscv_vals))]
    h_plugin = float(bw_selection.improved_sheather_jones(data))

    methods = {
        "MLKDE": best_h,
        "LSCV": h_lscv,
        "Plug-in (Sheather–Jones)": h_plugin,
    }
    summary = pd.DataFrame(
        {
            "metodo": list(methods.keys()),
            "h": [methods[m] for m in methods],
            "log_verossimilhanca_loo": [loo_log_likelihood(data, h) for h in methods.values()],
        }
    ).sort_values("log_verossimilhanca_loo", ascending=False)
    return summary, methods


comparacao_h, metodo_h = score_methods(log_prices, h_grid)
comparacao_h

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(log_prices, bins=40, density=True, alpha=0.3, color="gray", edgecolor="white", label="Histograma")
colors = {"MLKDE": "crimson", "LSCV": "navy", "Plug-in (Sheather–Jones)": "darkorange"}
for nome, h in metodo_h.items():
    ax.plot(x_grid, kde_density(x_grid, log_prices, h), color=colors[nome], linewidth=2, label=f"{nome} (h={h:.3f})")

ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade")
ax.set_title("KDE com diferentes escolhas de bandwidth")
ax.legend()
plt.show()

- O método com maior log-verossimilhança costuma ser o MLKDE (tabela acima). Ele premia curvas que tornam os dados observados mais prováveis.
- As curvas são parecidas, mas **LSCV** pode suavizar demais e o **Plug-in** tende a ser conservador; o MLKDE segue melhor os picos reais.

In [ ]:
point_densities = kde_density(log_prices, log_prices, h_mlkde)
limiar = np.percentile(point_densities, 1)
clean_df["densidade_mlkde"] = point_densities
clean_df["outlier"] = np.where(point_densities < limiar, "Outlier", "Faixa típica")

outliers = clean_df.sort_values("densidade_mlkde").head(8)[["price", "log_price", "densidade_mlkde", "outlier"]]
faixa_baixa, faixa_alta = np.percentile(clean_df["price"], [5, 95])

outliers

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(clean_df["price"], bins=50, color="lightgray", edgecolor="white")
ax.axvspan(faixa_baixa, faixa_alta, color="mediumseagreen", alpha=0.2, label="Faixa típica")
out_df = clean_df[clean_df["outlier"] == "Outlier"]
ax.scatter(out_df["price"], np.full(len(out_df), 0.0005), color="crimson", label="Outliers", zorder=3)
ax.set_xlabel("Preço por noite (EUR)")
ax.set_ylabel("Frequência")
ax.set_title("Preços originais com outliers destacados")
ax.legend()
plt.show()

fig, ax = plt.subplots(figsize=(7, 4))
ax.scatter(clean_df["log_price"], clean_df["densidade_mlkde"], alpha=0.4, color="steelblue", label="Anúncios")
ax.axhline(limiar, color="crimson", linestyle="--", label="Limiar (1º percentil)")
ax.scatter(out_df["log_price"], out_df["densidade_mlkde"], color="crimson", label="Outliers", zorder=3)
ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade estimada")
ax.set_title("Densidade MLKDE por anúncio")
ax.legend()
plt.show()

Outliers aparecem nas duas caudas: anúncios muito baratos (possíveis promoções ou erros) e anúncios muito caros (ofertas premium ou preços fora do padrão). Eles ficam em regiões de **densidade muito baixa** segundo o KDE.